In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
# Import netcdf-file
filename = '/lustre/storeB/immutable/archive/projects/metproduction/meps/2023/10/10/meps_lagged_6_h_subset_2_5km_20231010T00Z.nc'
data = xr.open_dataset(filename)

In [3]:
def find_nearest_coordinates(param, target_lat, target_lon):
    latitude_values = data['latitude'].values
    longitude_values = data['longitude'].values

    # Calculate the euclidean distance from the target coordinates
    distances = np.sqrt((latitude_values - target_lat)**2 + (longitude_values - target_lon)**2)

    # Find indices of the minimum distance
    nearest_index = np.unravel_index(distances.argmin(), distances.shape)

    # Extract corresponding y and x indices
    nearest_y_index = nearest_index[0]
    nearest_x_index = nearest_index[1]

    # Extract data using found indices
    print(f'Successfully extracted (y,x) for target coordinates')
    return nearest_y_index, nearest_x_index


In [4]:
def extract_vals_to_df(data_param, y_index, x_index):
    
    data_param = data_param.sel(   
        y = data['y'][y_index],   
        x = data['x'][x_index],
        method='nearest'
     )
    data_param = data_param.to_dataframe()
    return data_param

In [139]:
# Target lon and lat
# ENKB: Kristiansund Airport

target_latitude = 63.107166238
target_longitude =  7.822330044 

y_index, x_index = find_nearest_coordinates(data['air_temperature_2m'], target_latitude, target_longitude)
air_temp_2m = extract_vals_to_df(data['air_temperature_2m'], y_index, x_index)



Successfully extracted (y,x) for target coordinates


In [ ]:
air_temp_2m

<xarray.DataArray 'air_temperature_2m' (time: 62, height1: 1,
                                        ensemble_member: 30)>
array([[[281.6079 , 281.53802, ..., 280.9359 , 281.08856]],

       [[281.55322, 281.95032, ..., 281.1546 , 281.35168]],

       ...,

       [[283.41873, 282.70956, ..., 282.94016, 283.216  ]],

       [[283.3785 , 282.22818, ..., 282.83218, 283.0525 ]]], dtype=float32)
Coordinates:
  * time             (time) datetime64[ns] 2023-10-10 ... 2023-10-12T13:00:00
  * ensemble_member  (ensemble_member) int16 0 1 2 3 4 5 6 ... 24 25 26 27 28 29
  * height1          (height1) float32 2.0
    x                float32 -3.601e+05
    y                float32 -17.9
    longitude        float64 7.823
    latitude         float64 63.12
Attributes:
    long_name:      Screen level temperature (T2M)
    standard_name:  air_temperature
    units:          K
    grid_mapping:   projection_lambert

In [5]:
airport_location = pd.read_csv('data/airport_positions.csv')
airport_location

,airport_identifier,name,position
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)"
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)"
2,ENAS,NY-ALESUND/HAMNERABBEN RWY 12/30,"(7892,1187)"
3,ENAT,ALTA RWY 11/29,"(6997,2337)"
4,ENBJ,BJORNOYA,"(7450,1908)"
...,...,...,...
85,ENVD,VADSO RWY 08/26,"(7006,2984)"
86,ENWV,VALHALL A,"(5627,339)"
87,ENVR,VAEROY,"(6765,1272)"
88,ENXW,GRANE,"(5916,248)"


In [6]:
airport_location[['latitude', 'longitude']] = airport_location['position'].str.split(',', expand=True)
airport_location

,airport_identifier,name,position,latitude,longitude
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)",(6256,611)
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)",(6929,1614)
2,ENAS,NY-ALESUND/HAMNERABBEN RWY 12/30,"(7892,1187)",(7892,1187)
3,ENAT,ALTA RWY 11/29,"(6997,2337)",(6997,2337)
4,ENBJ,BJORNOYA,"(7450,1908)",(7450,1908)
...,...,...,...,...,...
85,ENVD,VADSO RWY 08/26,"(7006,2984)",(7006,2984)
86,ENWV,VALHALL A,"(5627,339)",(5627,339)
87,ENVR,VAEROY,"(6765,1272)",(6765,1272)
88,ENXW,GRANE,"(5916,248)",(5916,248)


In [7]:
# Remove remaining parenthesis
airport_location['latitude'] = airport_location['latitude'].str.replace('(', '')
airport_location['longitude'] = airport_location['longitude'].str.replace(')', '')
airport_location

/tmp/ipykernel_1608529/1383077076.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  airport_location['latitude'] = airport_location['latitude'].str.replace('(', '')
/tmp/ipykernel_1608529/1383077076.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  airport_location['longitude'] = airport_location['longitude'].str.replace(')', '')


,airport_identifier,name,position,latitude,longitude
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)",6256,611
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)",6929,1614
2,ENAS,NY-ALESUND/HAMNERABBEN RWY 12/30,"(7892,1187)",7892,1187
3,ENAT,ALTA RWY 11/29,"(6997,2337)",6997,2337
4,ENBJ,BJORNOYA,"(7450,1908)",7450,1908
...,...,...,...,...,...
85,ENVD,VADSO RWY 08/26,"(7006,2984)",7006,2984
86,ENWV,VALHALL A,"(5627,339)",5627,339
87,ENVR,VAEROY,"(6765,1272)",6765,1272
88,ENXW,GRANE,"(5916,248)",5916,248


In [8]:
# Dividing by 100 to get correct values for latitude and longitute
airport_location['latitude'] = airport_location['latitude'].astype('float')/100
airport_location['longitude'] = airport_location['longitude'].astype('float')/100
airport_location

,airport_identifier,name,position,latitude,longitude
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)",62.56,6.11
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)",69.29,16.14
2,ENAS,NY-ALESUND/HAMNERABBEN RWY 12/30,"(7892,1187)",78.92,11.87
3,ENAT,ALTA RWY 11/29,"(6997,2337)",69.97,23.37
4,ENBJ,BJORNOYA,"(7450,1908)",74.50,19.08
...,...,...,...,...,...
85,ENVD,VADSO RWY 08/26,"(7006,2984)",70.06,29.84
86,ENWV,VALHALL A,"(5627,339)",56.27,3.39
87,ENVR,VAEROY,"(6765,1272)",67.65,12.72
88,ENXW,GRANE,"(5916,248)",59.16,2.48


In [9]:
# Apply the functions to each row
def process_airport_row(row):
    target_lat = row['latitude']
    target_lon = row['longitude']

    nearest_y_index, nearest_x_index = find_nearest_coordinates(data['air_temperature_2m'], target_lat, target_lon)
    
    # Assuming 'data' is your dataset, replace it with your actual dataset
    result_df = extract_vals_to_df(data['air_temperature_2m'], nearest_y_index, nearest_x_index)
    
    return result_df

In [10]:
test_dataset = airport_location.head(2)
test_dataset

,airport_identifier,name,position,latitude,longitude
0,ENAL,ALESUND/VIGRA RWY 07/25,"(6256,611)",62.56,6.11
1,ENAN,ANDOYA/ANDENES RWY 03/21 14/32,"(6929,1614)",69.29,16.14


In [11]:
# Apply the process_airport_row function to each row of the DataFrame
result = test_dataset.apply(process_airport_row, axis=1)

# Concatenate the results into a final DataFrame
final_result = pd.concat(result.tolist(), keys=test_dataset['airport_identifier'])

# Print or use the final_result DataFrame as needed
print(final_result)

Successfully extracted (y,x) for target coordinates
Successfully extracted (y,x) for target coordinates
                                                                            x  \
airport_identifier time                height1 ensemble_member                  
ENAL               2023-10-10 00:00:00 2.0     0               -455084.062500   
                                               1               -455084.062500   
                                               2               -455084.062500   
                                               3               -455084.062500   
                                               4               -455084.062500   
...                                                                       ...   
ENAN               2023-10-12 13:00:00 2.0     25                44915.945312   
                                               26                44915.945312   
                                               27                44915.945312   
     

In [12]:
final_result

x  \
airport_identifier time                height1 ensemble_member                  
ENAL               2023-10-10 00:00:00 2.0     0               -455084.062500   
                                               1               -455084.062500   
                                               2               -455084.062500   
                                               3               -455084.062500   
                                               4               -455084.062500   
...                                                                       ...   
ENAN               2023-10-12 13:00:00 2.0     25                44915.945312   
                                               26                44915.945312   
                                               27                44915.945312   
                                               28                44915.945312   
                                               29                44915.945312   

                                                                           y  \
airport_identifier time                height1 ensemble_member                 
ENAL               2023-10-10 00:00:00 2.0     0                -50017.90625   
                                               1                -50017.90625   
                                               2                -50017.90625   
                                               3                -50017.90625   
                                               4                -50017.90625   
...                                                                      ...   
ENAN               2023-10-12 13:00:00 2.0     25               667482.12500   
                                               26               667482.12500   
                                               27               667482.12500   
                                               28               667482.12500   
                                               29               667482.12500   

                                                                longitude  \
airport_identifier time                height1 ensemble_member              
ENAL               2023-10-10 00:00:00 2.0     0                 6.089178   
                                               1                 6.089178   
                                               2                 6.089178   
                                               3                 6.089178   
                                               4                 6.089178   
...                                                                   ...   
ENAN               2023-10-12 13:00:00 2.0     25               16.135429   
                                               26               16.135429   
                                               27               16.135429   
                                               28               16.135429   
                                               29               16.135429   

                                                                 latitude  \
airport_identifier time                height1 ensemble_member              
ENAL               2023-10-10 00:00:00 2.0     0                62.565422   
                                               1                62.565422   
                                               2                62.565422   
                                               3                62.565422   
                                               4                62.565422   
...                                                                   ...   
ENAN               2023-10-12 13:00:00 2.0     25               69.287653   
                                               26               69.287653   
                                               27               69.287653   
                                               28               69.287653   
                                               29      

In [13]:
final_result.to_xarray()

<xarray.Dataset>
Dimensions:             (airport_identifier: 2, time: 62, height1: 1,
                         ensemble_member: 30)
Coordinates:
  * airport_identifier  (airport_identifier) object 'ENAL' 'ENAN'
  * time                (time) datetime64[ns] 2023-10-10 ... 2023-10-12T13:00:00
  * height1             (height1) float64 2.0
  * ensemble_member     (ensemble_member) int64 0 1 2 3 4 5 ... 25 26 27 28 29
Data variables:
    x                   (airport_identifier, time, height1, ensemble_member) float32 ...
    y                   (airport_identifier, time, height1, ensemble_member) float32 ...
    longitude           (airport_identifier, time, height1, ensemble_member) float64 ...
    latitude            (airport_identifier, time, height1, ensemble_member) float64 ...
    air_temperature_2m  (airport_identifier, time, height1, ensemble_member) float32 ...

In [119]:
metar_msg_total = pd.read_csv('data/metar_msg_total.csv')

In [120]:
metar_msg_total = metar_msg_total.sort_values(['issued_at', 'airport_identifier'])
metar_msg_total = metar_msg_total.reset_index(drop=True)

In [121]:
metar_msg_total

,airport_identifier,metar,issued_at
0,ENAL,ENAL 010020Z 08005KT 9999 SCT024/// BKN037/// ...,2021-01-01 00:20:00
1,ENAN,ENAN 010020Z 15016KT 9999 FEW092/// 03/M02 Q1011=,2021-01-01 00:20:00
2,ENAT,ENAT 010020Z 17006KT 9999 OVC022/// M01/M04 Q1...,2021-01-01 00:20:00
3,ENBN,ENBN 010020Z 08008KT 060V120 9999NDV OVC074///...,2021-01-01 00:20:00
4,ENBO,ENBO 010020Z 11013KT CAVOK 01/M06 Q1009=,2021-01-01 00:20:00
...,...,...,...
3261262,ENVD,ENVD 312350Z 29015KT 9999 FEW024/// BKN044/// ...,2023-12-31 23:50:00
3261263,ENWG,ENWG 312350Z 12047KT 9999 -SHRA OVC006/// 05/0...,2023-12-31 23:50:00
3261264,ENWV,ENWV 312350Z 14019KT 9999NDV SCT014/// OVC037/...,2023-12-31 23:50:00
3261265,ENXW,ENXW 312350Z 13041G54KT //// FEW002/// SCT014/...,2023-12-31 23:50:00


In [122]:
metar_msg_total['issued_at'] = pd.to_datetime(metar_msg_total['issued_at'])

In [123]:
metar_msg_total.to_xarray()

<xarray.Dataset>
Dimensions:             (index: 3261267)
Coordinates:
  * index               (index) int64 0 1 2 3 ... 3261264 3261265 3261266
Data variables:
    airport_identifier  (index) object 'ENAL' 'ENAN' 'ENAT' ... 'ENXW' 'ENZV'
    metar               (index) object 'ENAL 010020Z 08005KT 9999 SCT024/// B...
    issued_at           (index) datetime64[ns] 2021-01-01T00:20:00 ... 2023-1...

In [124]:
#metar_msg_total = metar_msg_total.rename(columns={'issued_at':'time'})
metar_msg_total["time"] = metar_msg_total.reset_index()["issued_at"].apply(lambda x: x.strftime("%Y-%m-%d %H:00:00"))
metar_msg_total['time'] = pd.to_datetime(metar_msg_total['time'])
metar_msg_total = metar_msg_total.set_index(['airport_identifier', 'time'])
metar_msg_total.sort_index(inplace=True)

In [126]:
metar_msg_total

metar  \
airport_identifier time                                                                     
ENAL               2021-01-01 00:00:00  ENAL 010020Z 08005KT 9999 SCT024/// BKN037/// ...   
                   2021-01-01 00:00:00  ENAL 010050Z 09006KT 9999 FEW025/// 00/M02 Q1006=   
                   2021-01-01 01:00:00  ENAL 010120Z 08005KT 9999 FEW027/// SCT040/// ...   
                   2021-01-01 01:00:00  ENAL 010150Z 09006KT 9999 BKN025/// M01/M02 Q1...   
                   2021-01-01 02:00:00  ENAL 010220Z 09007KT 9999 OVC026/// 01/M01 Q1006=   
...                                                                                   ...   
ENZV               2023-12-31 21:00:00           ENZV 312150Z 10022KT CAVOK 05/M05 Q0994=   
                   2023-12-31 22:00:00        ENZV 312220Z 10023G34KT CAVOK 05/M05 Q0994=   
                   2023-12-31 22:00:00           ENZV 312250Z 10021KT CAVOK 05/M04 Q0994=   
                   2023-12-31 23:00:00           ENZV 312320Z 10024KT CAVOK 04/M05 Q0994=   
                   2023-12-31 23:00:00        ENZV 312350Z 11025G37KT CAVOK 04/M05 Q0995=   

                                                 issued_at  
airport_identifier time                                     
ENAL               2021-01-01 00:00:00 2021-01-01 00:20:00  
                   2021-01-01 00:00:00 2021-01-01 00:50:00  
                   2021-01-01 01:00:00 2021-01-01 01:20:00  
                   2021-01-01 01:00:00 2021-01-01 01:50:00  
                   2021-01-01 02:00:00 2021-01-01 02:20:00  
...                                                    ...  
ENZV               2023-12-31 21:00:00 2023-12-31 21:50:00  
                   2023-12-31 22:00:00 2023-12-31 22:20:00  
                   2023-12-31 22:00:00 2023-12-31 22:50:00  
                   2023-12-31 23:00:00 2023-12-31 23:20:00  
                   2023-12-31 23:00:00 2023-12-31 23:50:00  

[3261267 rows x 2 columns]

In [ ]:
pd.concat([final_result.reset_index(["height1","ensemble_member"]),metar_msg_total],axis=1)

In [129]:
metar_msg_total.merge(final_result.reset_index(["height1","ensemble_member"]), left_index=True, right_index=True).head(100).to_csv("prout_tonje.csv")

In [29]:
merge=pd.merge(final_result,metar_msg_total, how='inner', on='airport_identifier')

In [30]:
merge

,airport_identifier,x,y,longitude,latitude,air_temperature_2m,metar,issued_at
0,ENAL,-455084.062500,-50017.90625,6.089178,62.565422,282.125488,ENAL 010020Z 08005KT 9999 SCT024/// BKN037/// ...,2021-01-01 00:20:00
1,ENAL,-455084.062500,-50017.90625,6.089178,62.565422,282.125488,ENAL 010050Z 09006KT 9999 FEW025/// 00/M02 Q1006=,2021-01-01 00:50:00
2,ENAL,-455084.062500,-50017.90625,6.089178,62.565422,282.125488,ENAL 010120Z 08005KT 9999 FEW027/// SCT040/// ...,2021-01-01 01:20:00
3,ENAL,-455084.062500,-50017.90625,6.089178,62.565422,282.125488,ENAL 010150Z 09006KT 9999 BKN025/// M01/M02 Q1...,2021-01-01 01:50:00
4,ENAL,-455084.062500,-50017.90625,6.089178,62.565422,282.125488,ENAL 010220Z 09007KT 9999 OVC026/// 01/M01 Q1006=,2021-01-01 02:20:00
...,...,...,...,...,...,...,...,...
196213255,ENAN,44915.945312,667482.12500,16.135429,69.287653,279.240967,ENAN 312150Z 20011KT 9999 NCD M07/M11 Q1020=,2023-12-31 21:50:00
196213256,ENAN,44915.945312,667482.12500,16.135429,69.287653,279.240967,ENAN 312220Z 19011KT 9999 NCD M07/M11 Q1020=,2023-12-31 22:20:00
196213257,ENAN,44915.945312,667482.12500,16.135429,69.287653,279.240967,ENAN 312250Z 19007KT 9999 NCD M07/M11 Q1020=,2023-12-31 22:50:00
196213258,ENAN,44915.945312,667482.12500,16.135429,69.287653,279.240967,ENAN 312320Z 20009KT 9999 NCD M07/M11 Q1020=,2023-12-31 23:20:00
